# COVID-19 Case Counts from Johns Hopkins University

**[Work in progress]**

This notebook creates a .csv file with cummulative confimed cases and deaths for ingestion into the Knowledge Graph.

Data source: [COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University](https://github.com/CSSEGISandData/COVID-19)

Author: Peter Rose (pwrose@ucsd.edu)

In [1]:
import os
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
from pathlib import Path
import pandas as pd
import dateutil

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_HOME = Path(os.getenv('NEO4J_HOME'))
print(NEO4J_HOME)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3


In [4]:
def split_by_day(df, day, label):
    day_df = df[['stateFips', 'countyFips', day]].copy()
    day_df.rename(columns={day: label}, inplace=True)
    day_df['date'] = day
    return day_df

### Process cummulative confirmed cases

In [5]:
cases = pd.read_csv("https://coronadatascraper.com/timeseries-tidy.csv.zip",  dtype='str')

In [6]:
cases = cases.fillna('')

In [7]:
cases.head()

,name,level,city,county,state,country,population,lat,long,aggregate,tz,date,type,value
0,"Antwerp, Flanders, Belgium",county,,Antwerp,Flanders,Belgium,1847486,51.2485,4.717499999999999,,Europe/Brussels,2020-01-22,cases,4
1,"Antwerp, Flanders, Belgium",county,,Antwerp,Flanders,Belgium,1847486,51.2485,4.717499999999999,,Europe/Brussels,2020-01-23,cases,4
2,"Antwerp, Flanders, Belgium",county,,Antwerp,Flanders,Belgium,1847486,51.2485,4.717499999999999,,Europe/Brussels,2020-01-23,growthFactor,1
3,"Antwerp, Flanders, Belgium",county,,Antwerp,Flanders,Belgium,1847486,51.2485,4.717499999999999,,Europe/Brussels,2020-01-24,cases,4
4,"Antwerp, Flanders, Belgium",county,,Antwerp,Flanders,Belgium,1847486,51.2485,4.717499999999999,,Europe/Brussels,2020-01-24,growthFactor,1
